# Mentor Matching Tool

In [0]:
import csv
import numpy as np
import cvxpy as cvx
import pandas as pd

We'll want to see all of the tables we display.

In [0]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Data Importation

If you're using Google Collab, use the following to upload your files. If you aren't, keep your files in the same directory the .ipynb

In [243]:
try:
    from google.colab import files
    uploaded = files.upload()
except:
    print("Not on Google Collab")


## Change `demo` to use a real CSV

If you're just demoing the mentor matching tool run this cell to use my example schedules.

In [0]:
demo = False

if demo:
    school_availability = np.array([
        [1, 1, 0, 0],
        [0, 1, 1, 1]
    ])

    school_names = ["Hayward", "Albany"]

    mentor_availability = np.array([
        [1, 1, 1, 0],
        [0, 1, 1, 0],
        [1, 1, 1, 1],
    ])
    mentor_names = ["Alan", "Kirsten", "Sophia"]

If you're actually doing this, you'll need to import the information from a CSV. We choose to omit some schools from our data because they don't provide enough information and add aditional constraints. They'll have to be matched by hand.

In [245]:
if not demo:
    SCHOOL_AVAILABILITIES_NAME = "school_availabilities (1).csv"
    
    with open(SCHOOL_AVAILABILITIES_NAME) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        school_names = next(csv_reader)

    df = pd.read_csv(SCHOOL_AVAILABILITIES_NAME)
    school_availability = df.values.T

    delete_names = ['John Henry High School', 'Salesian College Preparatory', 'Albany High School']
    delete_indices = [school_names.index(name) for name in delete_names]
    print(school_availability.shape)
    school_availability = np.delete(school_availability, delete_indices, axis=0)
    print(school_availability.shape)
    school_names = np.delete(school_names, delete_indices, axis=0)
    
    school_availability = school_availability[1:]
    school_names = school_names[1:]

(17, 70)
(14, 70)


You must manually edit the `time_slot_names`, particularly if `demo is True`.

In [0]:
time_slot_names = ['Mon 10:00 - 11:00', 'Mon 11:00 - 12:00', 'Mon 12:00 - 1:00',
       'Mon 1:00 - 2:00', 'Mon 2:00 - 3:00', 'Mon 3:00 - 4:00',
       'Mon 4:00 - 5:00', 'Mon 5:00 - 6:00', 'Mon 6:00 - 7:00',
       'Mon 7:00 - 8:00', 'Tue 10:00 - 11:00', 'Tue 11:00 - 12:00',
       'Tue 12:00 - 1:00', 'Tue 1:00 - 2:00', 'Tue 2:00 - 3:00',
       'Tue 3:00 - 4:00', 'Tue 4:00 - 5:00', 'Tue 5:00 - 6:00',
       'Tue 6:00 - 7:00', 'Tue 7:00 - 8:00', 'Wed 10:00 - 11:00',
       'Wed 11:00 - 12:00', 'Wed 12:00 - 1:00', 'Wed 1:00 - 2:00',
       'Wed 2:00 - 3:00', 'Wed 3:00 - 4:00', 'Wed 4:00 - 5:00',
       'Wed 5:00 - 6:00', 'Wed 6:00 - 7:00', 'Wed 7:00 - 8:00',
       'Thu 10:00 - 11:00', 'Thu 11:00 - 12:00', 'Thu 12:00 - 1:00',
       'Thu 1:00 - 2:00', 'Thu 2:00 - 3:00', 'Thu 3:00 - 4:00',
       'Thu 4:00 - 5:00', 'Thu 5:00 - 6:00', 'Thu 6:00 - 7:00',
       'Thu 7:00 - 8:00', 'Fri 10:00 - 11:00', 'Fri 11:00 - 12:00',
       'Fri 12:00 - 1:00', 'Fri 1:00 - 2:00', 'Fri 2:00 - 3:00',
       'Fri 3:00 - 4:00', 'Fri 4:00 - 5:00', 'Fri 5:00 - 6:00',
       'Fri 6:00 - 7:00', 'Fri 7:00 - 8:00', 'Sat 10:00 - 11:00',
       'Sat 11:00 - 12:00', 'Sat 12:00 - 1:00', 'Sat 1:00 - 2:00',
       'Sat 2:00 - 3:00', 'Sat 3:00 - 4:00', 'Sat 4:00 - 5:00',
       'Sat 5:00 - 6:00', 'Sat 6:00 - 7:00', 'Sat 7:00 - 8:00',
       'Sun 10:00 - 11:00', 'Sun 11:00 - 12:00', 'Sun 12:00 - 1:00',
       'Sun 1:00 - 2:00', 'Sun 2:00 - 3:00', 'Sun 3:00 - 4:00',
       'Sun 4:00 - 5:00', 'Sun 5:00 - 6:00', 'Sun 6:00 - 7:00',
       'Sun 7:00 - 8:00']
num_time_slots = len(time_slot_names)

In [247]:
pd.DataFrame(school_availability, index=school_names, columns=time_slot_names)

,Mon 10:00 - 11:00,Mon 11:00 - 12:00,Mon 12:00 - 1:00,Mon 1:00 - 2:00,Mon 2:00 - 3:00,Mon 3:00 - 4:00,Mon 4:00 - 5:00,Mon 5:00 - 6:00,Mon 6:00 - 7:00,Mon 7:00 - 8:00,Tue 10:00 - 11:00,Tue 11:00 - 12:00,Tue 12:00 - 1:00,Tue 1:00 - 2:00,Tue 2:00 - 3:00,Tue 3:00 - 4:00,Tue 4:00 - 5:00,Tue 5:00 - 6:00,Tue 6:00 - 7:00,Tue 7:00 - 8:00,Wed 10:00 - 11:00,Wed 11:00 - 12:00,Wed 12:00 - 1:00,Wed 1:00 - 2:00,Wed 2:00 - 3:00,Wed 3:00 - 4:00,Wed 4:00 - 5:00,Wed 5:00 - 6:00,Wed 6:00 - 7:00,Wed 7:00 - 8:00,Thu 10:00 - 11:00,Thu 11:00 - 12:00,Thu 12:00 - 1:00,Thu 1:00 - 2:00,Thu 2:00 - 3:00,Thu 3:00 - 4:00,Thu 4:00 - 5:00,Thu 5:00 - 6:00,Thu 6:00 - 7:00,Thu 7:00 - 8:00,Fri 10:00 - 11:00,Fri 11:00 - 12:00,Fri 12:00 - 1:00,Fri 1:00 - 2:00,Fri 2:00 - 3:00,Fri 3:00 - 4:00,Fri 4:00 - 5:00,Fri 5:00 - 6:00,Fri 6:00 - 7:00,Fri 7:00 - 8:00,Sat 10:00 - 11:00,Sat 11:00 - 12:00,Sat 12:00 - 1:00,Sat 1:00 - 2:00,Sat 2:00 - 3:00,Sat 3:00 - 4:00,Sat 4:00 - 5:00,Sat 5:00 - 6:00,Sat 6:00 - 7:00,Sat 7:00 - 8:00,Sun 10:00 - 11:00,Sun 11:00 - 12:00,Sun 12:00 - 1:00,Sun 1:00 - 2:00,Sun 2:00 - 3:00,Sun 3:00 - 4:00,Sun 4:00 - 5:00,Sun 5:00 - 6:00,Sun 6:00 - 7:00,Sun 7:00 - 8:00
Middle College High School,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ACLC,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Pinole valley high,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0
Hercules High School,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
LPS Richmond,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Oakland School for the Arts,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
Skyline,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
El Cerrito High School,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
REALM Charter School,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Arroyo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [248]:
if not demo:
    MENTOR_AVAILABILITIES_NAME = "mentor_availabilities (1).csv"
  
    with open(MENTOR_AVAILABILITIES_NAME) as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      mentor_names = next(csv_reader)
      
    df = pd.read_csv(MENTOR_AVAILABILITIES_NAME)
    mentor_availability = df.values.T
    
    delete_names = ['Siying']
    delete_indices = [mentor_names.index(name) for name in delete_names]
    print(mentor_availability.shape)
    mentor_availability = np.delete(mentor_availability, delete_indices, axis=0)
    print(mentor_availability.shape)
    mentor_names = np.delete(mentor_names, delete_indices, axis=0)
    
    mentor_availability = mentor_availability[1:]
    mentor_names = mentor_names[1:]

(33, 70)
(32, 70)


In [249]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.DataFrame(mentor_availability, index=mentor_names, columns=time_slot_names))

,Mon 10:00 - 11:00,Mon 11:00 - 12:00,Mon 12:00 - 1:00,Mon 1:00 - 2:00,Mon 2:00 - 3:00,Mon 3:00 - 4:00,Mon 4:00 - 5:00,Mon 5:00 - 6:00,Mon 6:00 - 7:00,Mon 7:00 - 8:00,Tue 10:00 - 11:00,Tue 11:00 - 12:00,Tue 12:00 - 1:00,Tue 1:00 - 2:00,Tue 2:00 - 3:00,Tue 3:00 - 4:00,Tue 4:00 - 5:00,Tue 5:00 - 6:00,Tue 6:00 - 7:00,Tue 7:00 - 8:00,Wed 10:00 - 11:00,Wed 11:00 - 12:00,Wed 12:00 - 1:00,Wed 1:00 - 2:00,Wed 2:00 - 3:00,Wed 3:00 - 4:00,Wed 4:00 - 5:00,Wed 5:00 - 6:00,Wed 6:00 - 7:00,Wed 7:00 - 8:00,Thu 10:00 - 11:00,Thu 11:00 - 12:00,Thu 12:00 - 1:00,Thu 1:00 - 2:00,Thu 2:00 - 3:00,Thu 3:00 - 4:00,Thu 4:00 - 5:00,Thu 5:00 - 6:00,Thu 6:00 - 7:00,Thu 7:00 - 8:00,Fri 10:00 - 11:00,Fri 11:00 - 12:00,Fri 12:00 - 1:00,Fri 1:00 - 2:00,Fri 2:00 - 3:00,Fri 3:00 - 4:00,Fri 4:00 - 5:00,Fri 5:00 - 6:00,Fri 6:00 - 7:00,Fri 7:00 - 8:00,Sat 10:00 - 11:00,Sat 11:00 - 12:00,Sat 12:00 - 1:00,Sat 1:00 - 2:00,Sat 2:00 - 3:00,Sat 3:00 - 4:00,Sat 4:00 - 5:00,Sat 5:00 - 6:00,Sat 6:00 - 7:00,Sat 7:00 - 8:00,Sun 10:00 - 11:00,Sun 11:00 - 12:00,Sun 12:00 - 1:00,Sun 1:00 - 2:00,Sun 2:00 - 3:00,Sun 3:00 - 4:00,Sun 4:00 - 5:00,Sun 5:00 - 6:00,Sun 6:00 - 7:00,Sun 7:00 - 8:00
Mighty,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1
Sophia,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Kirsten,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,0,0,0,1,1,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1
Cameron,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0
Precious,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
Xiaowei,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1
Ryan,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alan,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,0,0,0,0,0,1,1,0,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0
Starr,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,0,1,1,1,1,0,1,1
Samantha,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [250]:
print("school names", school_names)
print("mentor names", mentor_names)


num_mentors = len(mentor_names)
num_schools = len(school_names)

school names ['Middle College High School' 'ACLC' 'Pinole valley high'
 'Hercules High School' 'LPS Richmond' 'Oakland School for the Arts'
 'Skyline' 'El Cerrito High School' 'REALM Charter School' 'Arroyo'
 "Bishop O'Dowd" 'Hayward High School' 'Miramonte High School']
mentor names ['Mighty' 'Sophia' 'Kirsten' 'Cameron' 'Precious' 'Xiaowei' 'Ryan' 'Alan'
 'Starr' 'Samantha' 'Sam' 'Matthew (Matt)' 'Chong' 'Conor' 'Deep'
 'Michael' 'Kevin' 'Yailin' 'Daniel' 'Bryant' 'Aaron' 'Vera' 'Alfredo'
 'Patrick' 'Natalie' 'Nur Zulaika (Zulaika)' 'Stephanie' 'Isaac' 'Juan'
 'Ifechukwu' 'Cindy']


In [0]:
def has_two_hour_block(a):
    for i in range(len(a) - 1):
        if a[i] and a[i + 1]:
            return int(1)
    return int(0)

compatability = np.zeros((num_schools, num_mentors))
for school_index in range(num_schools):
    for mentor_index in range(num_mentors):
        mutual_availability = school_availability[school_index] * mentor_availability[mentor_index]
        compatability[school_index, mentor_index] = has_two_hour_block(mutual_availability)


Do all mentors have at least one matching school?

In [252]:
print(compatability.any(axis=0).all())
print(compatability.any(axis=0))

True
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]


Do all schools have at least one matching mentor?

In [253]:
print(compatability.any(axis=1).all())
print(compatability.any(axis=1))

True
[ True  True  True  True  True  True  True  True  True  True  True  True
  True]


In [254]:
pd.DataFrame(compatability, index=school_names, columns=mentor_names).astype(int)

,Mighty,Sophia,Kirsten,Cameron,Precious,Xiaowei,Ryan,Alan,Starr,Samantha,Sam,Matthew (Matt),Chong,Conor,Deep,Michael,Kevin,Yailin,Daniel,Bryant,Aaron,Vera,Alfredo,Patrick,Natalie,Nur Zulaika (Zulaika),Stephanie,Isaac,Juan,Ifechukwu,Cindy
Middle College High School,1,0,1,0,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0
ACLC,1,1,1,1,0,0,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0
Pinole valley high,1,0,1,1,0,1,0,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1
Hercules High School,1,0,1,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1
LPS Richmond,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0
Oakland School for the Arts,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Skyline,1,0,1,1,1,1,0,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,0,1,0,0,0,1
El Cerrito High School,1,1,1,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
REALM Charter School,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Arroyo,1,1,1,0,0,0,1,1,1,1,0,1,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0


A value of 1 at `assignments[i][j]` means that the ith school and jth mentor are matched.

In [0]:
if True:
    assignments = cvx.Variable((num_schools, num_mentors), boolean=True)
else:
    assignments = [[0 for i in range(num_mentors)] for j in range(num_schools)]
    for school in range(num_schools):
      for mentor in range(num_mentors):
        assignments[school][mentor] = cvx.Variable(boolean=True, name="a" + f"{school_names[school]} - {mentor_names[mentor]}")



# Constraints

We now implement the constraints.

All schools must have at least one mentor. All schools must have no more than 3 mentors.

In [0]:
constraints = []
for school_index in range(num_schools):
    constraints.append(sum(assignments[school_index]) >= 1)
    constraints.append(sum(assignments[school_index]) <= 3)

All mentors must have exactly one school.

In [0]:
for mentor_index in range(num_mentors):
    num_schools_for_mentor = 0
    for school_index in range(num_schools): 
        num_schools_for_mentor += assignments[school_index][mentor_index]
    constraints.append(num_schools_for_mentor == 1)

A mentor who is a assigned to a school must have at least a 2 hour block of availability in common with the school

In [0]:
for mentor_index in range(num_mentors):
    for school_index in range(num_schools): #num_schools):

        a = assignments[school_index][mentor_index]
        c = compatability[school_index, mentor_index]
        # not assigned or is compatible
        condition = (1-a) + c >= 1
        constraints.append(condition)

# Optimization

Each possible answer for the pairing of school and mentors has a score. The score is the sum of the point values for each of the mentor-school assignments. The algorithm seeks to maximize the score without violating any constraints.

By default, each mentor-school assignment is worth 100 points. Assigning Sophia to Hayward would increase the score by 100 points. If we want to encourage the algorithm to make a particular assignment, we can increase its score.

    > weights["Alan"]["Hayward"] = 500
    
If we want to always have the algorithm choose a certain assignment, we can assign it a point value of a large number like $10^{10}$. The algorithm will choose this assignment unless it violates constraints or there are alternatives with equally high point values.

In future versions, we should account for mentor preferenes, mentor confidences, etc.

In [259]:
BIG = 10**10

weights = pd.DataFrame(
    np.ones((num_schools, num_mentors)) * 100,
    index=school_names,
    columns=mentor_names)
weights

,Mighty,Sophia,Kirsten,Cameron,Precious,Xiaowei,Ryan,Alan,Starr,Samantha,Sam,Matthew (Matt),Chong,Conor,Deep,Michael,Kevin,Yailin,Daniel,Bryant,Aaron,Vera,Alfredo,Patrick,Natalie,Nur Zulaika (Zulaika),Stephanie,Isaac,Juan,Ifechukwu,Cindy
Middle College High School,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
ACLC,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Pinole valley high,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Hercules High School,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
LPS Richmond,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Oakland School for the Arts,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Skyline,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
El Cerrito High School,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
REALM Charter School,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Arroyo,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [0]:
# More weight adjustments here
# weights["Alan"]["Hayward"] = 500

In [0]:
score = cvx.sum(cvx.multiply(weights, assignments))
score = 100 * assignments[0, 0] + 3
score = assignments[0, 0]

# Solving and Results

In [264]:
objective = cvx.Maximize(score)
problem = cvx.Problem(objective, constraints)

score = problem.solve()
print("score: ", score)

score:  nan


In [265]:
pd.DataFrame(
    assignments.value > 0.5,
    index=school_names,
    columns=mentor_names
)

,Mighty,Sophia,Kirsten,Cameron,Precious,Xiaowei,Ryan,Alan,Starr,Samantha,Sam,Matthew (Matt),Chong,Conor,Deep,Michael,Kevin,Yailin,Daniel,Bryant,Aaron,Vera,Alfredo,Patrick,Natalie,Nur Zulaika (Zulaika),Stephanie,Isaac,Juan,Ifechukwu,Cindy
Middle College High School,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False
ACLC,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
Pinole valley high,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False
Hercules High School,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
LPS Richmond,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
Oakland School for the Arts,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
Skyline,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True
El Cerrito High School,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
REALM Charter School,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Arroyo,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


# Verification

We print here the mentors for each school and each mentors availability with that school.

In [266]:
for school_index in range(num_schools):
    school_name = school_names[school_index]
    print()
    print(school_name)
    school_mentor_names = [school_name]
    mutual_availabilities = [school_availability[school_index]]
    for mentor_index in range(num_mentors):
        if assignments[school_index][mentor_index].value >= 0.5:
            school_mentor_names.append(mentor_names[mentor_index])
            mutual_availability = mentor_availability[mentor_index] * school_availability[school_index]
            mutual_availabilities.append(mutual_availability)
    combo = np.vstack(mutual_availabilities)
    display(pd.DataFrame(combo, index=school_mentor_names, columns=time_slot_names))


Middle College High School


,Mon 10:00 - 11:00,Mon 11:00 - 12:00,Mon 12:00 - 1:00,Mon 1:00 - 2:00,Mon 2:00 - 3:00,Mon 3:00 - 4:00,Mon 4:00 - 5:00,Mon 5:00 - 6:00,Mon 6:00 - 7:00,Mon 7:00 - 8:00,Tue 10:00 - 11:00,Tue 11:00 - 12:00,Tue 12:00 - 1:00,Tue 1:00 - 2:00,Tue 2:00 - 3:00,Tue 3:00 - 4:00,Tue 4:00 - 5:00,Tue 5:00 - 6:00,Tue 6:00 - 7:00,Tue 7:00 - 8:00,Wed 10:00 - 11:00,Wed 11:00 - 12:00,Wed 12:00 - 1:00,Wed 1:00 - 2:00,Wed 2:00 - 3:00,Wed 3:00 - 4:00,Wed 4:00 - 5:00,Wed 5:00 - 6:00,Wed 6:00 - 7:00,Wed 7:00 - 8:00,Thu 10:00 - 11:00,Thu 11:00 - 12:00,Thu 12:00 - 1:00,Thu 1:00 - 2:00,Thu 2:00 - 3:00,Thu 3:00 - 4:00,Thu 4:00 - 5:00,Thu 5:00 - 6:00,Thu 6:00 - 7:00,Thu 7:00 - 8:00,Fri 10:00 - 11:00,Fri 11:00 - 12:00,Fri 12:00 - 1:00,Fri 1:00 - 2:00,Fri 2:00 - 3:00,Fri 3:00 - 4:00,Fri 4:00 - 5:00,Fri 5:00 - 6:00,Fri 6:00 - 7:00,Fri 7:00 - 8:00,Sat 10:00 - 11:00,Sat 11:00 - 12:00,Sat 12:00 - 1:00,Sat 1:00 - 2:00,Sat 2:00 - 3:00,Sat 3:00 - 4:00,Sat 4:00 - 5:00,Sat 5:00 - 6:00,Sat 6:00 - 7:00,Sat 7:00 - 8:00,Sun 10:00 - 11:00,Sun 11:00 - 12:00,Sun 12:00 - 1:00,Sun 1:00 - 2:00,Sun 2:00 - 3:00,Sun 3:00 - 4:00,Sun 4:00 - 5:00,Sun 5:00 - 6:00,Sun 6:00 - 7:00,Sun 7:00 - 8:00
Middle College High School,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Samantha,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bryant,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Ifechukwu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



ACLC


,Mon 10:00 - 11:00,Mon 11:00 - 12:00,Mon 12:00 - 1:00,Mon 1:00 - 2:00,Mon 2:00 - 3:00,Mon 3:00 - 4:00,Mon 4:00 - 5:00,Mon 5:00 - 6:00,Mon 6:00 - 7:00,Mon 7:00 - 8:00,Tue 10:00 - 11:00,Tue 11:00 - 12:00,Tue 12:00 - 1:00,Tue 1:00 - 2:00,Tue 2:00 - 3:00,Tue 3:00 - 4:00,Tue 4:00 - 5:00,Tue 5:00 - 6:00,Tue 6:00 - 7:00,Tue 7:00 - 8:00,Wed 10:00 - 11:00,Wed 11:00 - 12:00,Wed 12:00 - 1:00,Wed 1:00 - 2:00,Wed 2:00 - 3:00,Wed 3:00 - 4:00,Wed 4:00 - 5:00,Wed 5:00 - 6:00,Wed 6:00 - 7:00,Wed 7:00 - 8:00,Thu 10:00 - 11:00,Thu 11:00 - 12:00,Thu 12:00 - 1:00,Thu 1:00 - 2:00,Thu 2:00 - 3:00,Thu 3:00 - 4:00,Thu 4:00 - 5:00,Thu 5:00 - 6:00,Thu 6:00 - 7:00,Thu 7:00 - 8:00,Fri 10:00 - 11:00,Fri 11:00 - 12:00,Fri 12:00 - 1:00,Fri 1:00 - 2:00,Fri 2:00 - 3:00,Fri 3:00 - 4:00,Fri 4:00 - 5:00,Fri 5:00 - 6:00,Fri 6:00 - 7:00,Fri 7:00 - 8:00,Sat 10:00 - 11:00,Sat 11:00 - 12:00,Sat 12:00 - 1:00,Sat 1:00 - 2:00,Sat 2:00 - 3:00,Sat 3:00 - 4:00,Sat 4:00 - 5:00,Sat 5:00 - 6:00,Sat 6:00 - 7:00,Sat 7:00 - 8:00,Sun 10:00 - 11:00,Sun 11:00 - 12:00,Sun 12:00 - 1:00,Sun 1:00 - 2:00,Sun 2:00 - 3:00,Sun 3:00 - 4:00,Sun 4:00 - 5:00,Sun 5:00 - 6:00,Sun 6:00 - 7:00,Sun 7:00 - 8:00
ACLC,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Sophia,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Deep,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Daniel,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



Pinole valley high


...

,Mon 10:00 - 11:00,Mon 11:00 - 12:00,Mon 12:00 - 1:00,Mon 1:00 - 2:00,Mon 2:00 - 3:00,Mon 3:00 - 4:00,Mon 4:00 - 5:00,Mon 5:00 - 6:00,Mon 6:00 - 7:00,Mon 7:00 - 8:00,Tue 10:00 - 11:00,Tue 11:00 - 12:00,Tue 12:00 - 1:00,Tue 1:00 - 2:00,Tue 2:00 - 3:00,Tue 3:00 - 4:00,Tue 4:00 - 5:00,Tue 5:00 - 6:00,Tue 6:00 - 7:00,Tue 7:00 - 8:00,Wed 10:00 - 11:00,Wed 11:00 - 12:00,Wed 12:00 - 1:00,Wed 1:00 - 2:00,Wed 2:00 - 3:00,Wed 3:00 - 4:00,Wed 4:00 - 5:00,Wed 5:00 - 6:00,Wed 6:00 - 7:00,Wed 7:00 - 8:00,Thu 10:00 - 11:00,Thu 11:00 - 12:00,Thu 12:00 - 1:00,Thu 1:00 - 2:00,Thu 2:00 - 3:00,Thu 3:00 - 4:00,Thu 4:00 - 5:00,Thu 5:00 - 6:00,Thu 6:00 - 7:00,Thu 7:00 - 8:00,Fri 10:00 - 11:00,Fri 11:00 - 12:00,Fri 12:00 - 1:00,Fri 1:00 - 2:00,Fri 2:00 - 3:00,Fri 3:00 - 4:00,Fri 4:00 - 5:00,Fri 5:00 - 6:00,Fri 6:00 - 7:00,Fri 7:00 - 8:00,Sat 10:00 - 11:00,Sat 11:00 - 12:00,Sat 12:00 - 1:00,Sat 1:00 - 2:00,Sat 2:00 - 3:00,Sat 3:00 - 4:00,Sat 4:00 - 5:00,Sat 5:00 - 6:00,Sat 6:00 - 7:00,Sat 7:00 - 8:00,Sun 10:00 - 11:00,Sun 11:00 - 12:00,Sun 12:00 - 1:00,Sun 1:00 - 2:00,Sun 2:00 - 3:00,Sun 3:00 - 4:00,Sun 4:00 - 5:00,Sun 5:00 - 6:00,Sun 6:00 - 7:00,Sun 7:00 - 8:00
Bishop O'Dowd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Juan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



Hayward High School


,Mon 10:00 - 11:00,Mon 11:00 - 12:00,Mon 12:00 - 1:00,Mon 1:00 - 2:00,Mon 2:00 - 3:00,Mon 3:00 - 4:00,Mon 4:00 - 5:00,Mon 5:00 - 6:00,Mon 6:00 - 7:00,Mon 7:00 - 8:00,Tue 10:00 - 11:00,Tue 11:00 - 12:00,Tue 12:00 - 1:00,Tue 1:00 - 2:00,Tue 2:00 - 3:00,Tue 3:00 - 4:00,Tue 4:00 - 5:00,Tue 5:00 - 6:00,Tue 6:00 - 7:00,Tue 7:00 - 8:00,Wed 10:00 - 11:00,Wed 11:00 - 12:00,Wed 12:00 - 1:00,Wed 1:00 - 2:00,Wed 2:00 - 3:00,Wed 3:00 - 4:00,Wed 4:00 - 5:00,Wed 5:00 - 6:00,Wed 6:00 - 7:00,Wed 7:00 - 8:00,Thu 10:00 - 11:00,Thu 11:00 - 12:00,Thu 12:00 - 1:00,Thu 1:00 - 2:00,Thu 2:00 - 3:00,Thu 3:00 - 4:00,Thu 4:00 - 5:00,Thu 5:00 - 6:00,Thu 6:00 - 7:00,Thu 7:00 - 8:00,Fri 10:00 - 11:00,Fri 11:00 - 12:00,Fri 12:00 - 1:00,Fri 1:00 - 2:00,Fri 2:00 - 3:00,Fri 3:00 - 4:00,Fri 4:00 - 5:00,Fri 5:00 - 6:00,Fri 6:00 - 7:00,Fri 7:00 - 8:00,Sat 10:00 - 11:00,Sat 11:00 - 12:00,Sat 12:00 - 1:00,Sat 1:00 - 2:00,Sat 2:00 - 3:00,Sat 3:00 - 4:00,Sat 4:00 - 5:00,Sat 5:00 - 6:00,Sat 6:00 - 7:00,Sat 7:00 - 8:00,Sun 10:00 - 11:00,Sun 11:00 - 12:00,Sun 12:00 - 1:00,Sun 1:00 - 2:00,Sun 2:00 - 3:00,Sun 3:00 - 4:00,Sun 4:00 - 5:00,Sun 5:00 - 6:00,Sun 6:00 - 7:00,Sun 7:00 - 8:00
Hayward High School,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Starr,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



Miramonte High School


,Mon 10:00 - 11:00,Mon 11:00 - 12:00,Mon 12:00 - 1:00,Mon 1:00 - 2:00,Mon 2:00 - 3:00,Mon 3:00 - 4:00,Mon 4:00 - 5:00,Mon 5:00 - 6:00,Mon 6:00 - 7:00,Mon 7:00 - 8:00,Tue 10:00 - 11:00,Tue 11:00 - 12:00,Tue 12:00 - 1:00,Tue 1:00 - 2:00,Tue 2:00 - 3:00,Tue 3:00 - 4:00,Tue 4:00 - 5:00,Tue 5:00 - 6:00,Tue 6:00 - 7:00,Tue 7:00 - 8:00,Wed 10:00 - 11:00,Wed 11:00 - 12:00,Wed 12:00 - 1:00,Wed 1:00 - 2:00,Wed 2:00 - 3:00,Wed 3:00 - 4:00,Wed 4:00 - 5:00,Wed 5:00 - 6:00,Wed 6:00 - 7:00,Wed 7:00 - 8:00,Thu 10:00 - 11:00,Thu 11:00 - 12:00,Thu 12:00 - 1:00,Thu 1:00 - 2:00,Thu 2:00 - 3:00,Thu 3:00 - 4:00,Thu 4:00 - 5:00,Thu 5:00 - 6:00,Thu 6:00 - 7:00,Thu 7:00 - 8:00,Fri 10:00 - 11:00,Fri 11:00 - 12:00,Fri 12:00 - 1:00,Fri 1:00 - 2:00,Fri 2:00 - 3:00,Fri 3:00 - 4:00,Fri 4:00 - 5:00,Fri 5:00 - 6:00,Fri 6:00 - 7:00,Fri 7:00 - 8:00,Sat 10:00 - 11:00,Sat 11:00 - 12:00,Sat 12:00 - 1:00,Sat 1:00 - 2:00,Sat 2:00 - 3:00,Sat 3:00 - 4:00,Sat 4:00 - 5:00,Sat 5:00 - 6:00,Sat 6:00 - 7:00,Sat 7:00 - 8:00,Sun 10:00 - 11:00,Sun 11:00 - 12:00,Sun 12:00 - 1:00,Sun 1:00 - 2:00,Sun 2:00 - 3:00,Sun 3:00 - 4:00,Sun 4:00 - 5:00,Sun 5:00 - 6:00,Sun 6:00 - 7:00,Sun 7:00 - 8:00
Miramonte High School,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0
Aaron,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,0
Vera,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,1,0,0,0,0,1,1,0,0,0,0,0
Patrick,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,0
